In [1]:
# import the necessary packages
import numpy as np
from imutils import face_utils
import dlib
import cv2

from locations import landmarks

In [2]:
# initialize dlib's face detector and create predictor
p = "../data/shape_predictor_194_face_landmarks.dat"
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(p)

In [3]:
# Create video reader and find length
v_cap = cv2.VideoCapture('../data/aczrgyricp.mp4')
v_len = int(v_cap.get(cv2.CAP_PROP_FRAME_COUNT))

# pick n frames
n_frames = 30
sample = np.linspace(0, v_len - 1, n_frames).astype(int)

# by video index
feature_face_color = 0
feature_face_maxdiff = 0
feature_yanpi_face_diff = 0

# by frame index
face_color_avgs = []
face_color_stds = []
yanpi_face_avgs = []

for j in range(v_len):
    success = v_cap.grab()
    if j in sample:
        print('=========第{}帧========'.format(j))
        success, frame = v_cap.retrieve()
        if not success:
            continue
            
        # detect faces in the grayscale image
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        rects = detector(gray, 0)
        
        # if there are more than one face in a frame, just skip it
        if len(rects) == 1:
            rect = rects[0]
        else:
            continue
        
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)
        xia_lian = abs(shape[0][1] - shape[109][1])
        to_append_pnt = [[shape[0][0], int(shape[0][1] + xia_lian * 0.4)], 
                           [shape[134][0], int(shape[134][1] + xia_lian * 0.4)]]

        # part counter indexes
        r_eye_index = landmarks().get_polygons_index('R_EYE')
        l_eye_index = landmarks().get_polygons_index('L_EYE')
        xiaba_index = landmarks().get_polygons_index('XIABA')

        # part counter points
        l_eye = shape[l_eye_index]
        r_eye = shape[r_eye_index]
        xiaba = np.concatenate((shape[xiaba_index], to_append_pnt), axis=0)

        # make masks with coordinates
        mask_xiaba = np.zeros((frame.shape[0], frame.shape[1]))
        mask_eye = np.zeros((frame.shape[0], frame.shape[1]))
        
        cv2.fillPoly(mask_xiaba, [xiaba], 1)
        cv2.fillPoly(mask_eye, [l_eye, r_eye], 1)
        
        mask_xiaba = mask_xiaba.astype(np.bool)
        xiaba_out = np.zeros_like(gray)
        xiaba_out[mask_xiaba] = gray[mask_xiaba]
        mask_eye = mask_eye.astype(np.bool)
        eye_out = np.zeros_like(gray)
        eye_out[mask_eye] = gray[mask_eye]

        l_x, l_y, l_w, l_h = cv2.boundingRect(l_eye)
        l_eye_rectangle = gray[int(l_y-0.25*l_h):l_y+l_h, l_x:l_x+l_w]
        r_x, r_y, r_w, r_h = cv2.boundingRect(r_eye)
        r_eye_rectangle = gray[int(r_y-0.25*r_h):r_y+r_h, r_x:r_x+r_w]

        # calc feature
        face_color_avg = xiaba_out.sum() / mask_xiaba.sum()
        face_color_avgs.append(face_color_avg)
        face_color_stds.append(xiaba_out.std())
        yanpi_avg = (l_eye_rectangle.sum() + r_eye_rectangle.sum() - eye_out.sum()) / ((l_eye_rectangle.shape[0] + \
                r_eye_rectangle.shape[0]) * (l_eye_rectangle.shape[1] + r_eye_rectangle.shape[1]) - mask_eye.sum())
        yanpi_face_avgs.append(yanpi_avg)
        
feature_face_color = np.array(face_color_avgs).std() # 面部色度标准差
feature_face_maxdiff = max(face_color_stds) # 面部色度最大差异（最大标准差）
feature_yanpi_face_diff = np.array(yanpi_face_avgs).std() # 眼皮面部色度差的标准差

v_cap.release()
print(feature_face_color, feature_face_maxdiff, feature_yanpi_face_diff)

=========第0帧========
=========第10帧========
=========第20帧========
=========第30帧========
=========第41帧========
=========第51帧========
=========第61帧========
=========第72帧========
=========第82帧========
=========第92帧========
=========第103帧========
=========第113帧========
=========第123帧========
=========第134帧========
=========第144帧========
=========第154帧========
=========第164帧========
=========第175帧========
=========第185帧========
=========第195帧========
=========第206帧========
=========第216帧========
=========第226帧========
=========第237帧========
=========第247帧========
=========第257帧========
=========第268帧========
=========第278帧========
=========第288帧========
=========第299帧========
1.6881987081101535 7.272663990512217 1.308329214654524
